# Get data from the python client

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Swagger UI)[https://control.green-fusion.de/services/energy-management-backend/v1/api/doc#/]


## Inits

In [1]:
#imports
from energy_management_client import BackendPythonClient
import pandas as pd
import json
from datetime import datetime
import pprint  # For pretty-printing
import ast  
import os

In [2]:
# select your environment
environment="prod" # prod or dev

# Selector
if environment=="dev":
    backend_endpoint_path="https://dev.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="development"
elif environment=="prod":
    backend_endpoint_path="https://control.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="control"  
else:
    backend_endpoint_path=""
    realm_type=""
    print("select either prod or dev for the environment")

#test
#backend_endpoint_path, realm_type
if True:
    # pythpon client auth & login
    # init the client   
    client = BackendPythonClient(backend_endpoint=backend_endpoint_path,)
    
    #login
    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm=realm_type,
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

## Functions

### buildings

In [3]:
def get_buildings_df(client, customers):
    """
    Loop through all customers, fetch their buildings, and build a DataFrame where each row contains:
      - building_id, customerID (from customer object), address, city, atvise_display_name, base,
      - heatingType, heatingSurface, access_key, nickname, postal_code, gui_name,
      - coordinates, CustomerAlias, gfid,
      - LayoutID (extracted from modular_system)

    Usage:
        customers = client.customers.get_list_of_customers()
        df = get_buildings_df(client, customers)
        print(df.head())
    """
    try:
        data = []
        for c in customers:
            customer_id = c.id
            customer_name = getattr(c, 'name', None)
            client.configure(customer_id=customer_id)
            
            # Get buildings
            c_buildings = client.buildings.get_buildings()
            building_list = getattr(c_buildings, 'buildings', [])
            
            for b in building_list:
                building_id         = getattr(b, 'building_id', None)
                address             = getattr(b, 'address', None)
                city                = getattr(b, 'city', None)
                atvise_display_name = getattr(b, 'atvise_display_name', None)
                base                = getattr(b, 'base', None)
                heatingType         = getattr(b, 'heatingType', None)
                heatingSurface      = getattr(b, 'heatingSurface', None)
                access_key          = getattr(b, 'access_key', None)
                nickname            = getattr(b, 'nickname', None)
                postal_code         = getattr(b, 'postal_code', None)
                gui_name            = getattr(b, 'gui_name', None)
                coordinates         = getattr(b, 'coordinates', None)
                CustomerAlias       = getattr(b, 'CustomerAlias', None)
                gfid                = getattr(b, 'gfid', None)
                
                # Parse modular_system
                modular_system_str = getattr(b, 'modular_system', None)
                LayoutID = None
                if modular_system_str:
                    try:
                        modular_system = json.loads(modular_system_str)
                        LayoutID = modular_system.get('layoutID', None)
                    except json.JSONDecodeError:
                        LayoutID = None

                data.append({
                    "building_id": building_id,
                    "customerID": customer_id,  # <-- corrected here
                    "customer_name": customer_name,
                    "address": address,
                    "city": city,
                    "atvise_display_name": atvise_display_name,
                    "base": base,
                    "heatingType": heatingType,
                    "heatingSurface": heatingSurface,
                    "access_key": access_key,
                    "nickname": nickname,
                    "postal_code": postal_code,
                    "gui_name": gui_name,
                    "coordinates": coordinates,
                    "CustomerAlias": CustomerAlias,
                    "gfid": gfid,
                    "modular_system": modular_system,  
                    "LayoutID": LayoutID
                })

        return pd.DataFrame(data)

    except Exception as e:
        print("API Call Failed:", e)
        return pd.DataFrame()

### devices

In [4]:
def get_devices(client):
    """
    usage:
    list_device=get_devices(client)
    list(list_device)
    """
    devices_list=client.devices.get_devices()
    devices_list=list(devices_list)[0][1]
    device_dicts = [device.__dict__ for device in devices_list]
    df_device = pd.DataFrame(device_dicts)
    return df_device

### T sensor count 

In [5]:
def get_t_sensor_count(building_id):
    try:
        sensors = client.buildings.get_list_of_sensors_by_building_id(building_id)
        count = sum(
            1 for s in sensors
            if s.unit == '°C'
            and s.source == 'GREENBOX_MQTT'
            and (s.long_name is None or 'Wärmemengenzähler' not in s.long_name)
        )
        return count
    except Exception as e:
        # Optionally log the error
        return None


In [6]:
#get_t_sensor_count(1750)

In [7]:
#client.buildings.get_list_of_sensors_by_building_id(2382)

### sub component

In [8]:
def get_sub_component(building_id):
    #print(f"Fetching subsystems for building_id: {building_id}")  # Debug
    subsystems = client.buildings.get_sub_systems_by_building_id(building_id)
    
    subsystems_infos = {
        s.acronym: f"type : {s.type} sub_type : {s.sub_type}"
        for s in subsystems
    }
    return subsystems_infos

In [9]:
#client.buildings.get_sub_systems_by_building_id(2382) 
get_sub_component(765)

{'global': 'type : GLOBAL sub_type : NOT_SPECIFIED',
 'dh': 'type : DISTRICT_HEATING sub_type : NOT_SPECIFIED',
 'hc(n)': 'type : HEATING_CIRCUIT sub_type : NOT_SPECIFIED',
 'hw(n)': 'type : WARM_WATER sub_type : STORAGE_TANK'}

## get_buildings_df

In [10]:
# Get customers list (list of dicts)
customers = client.customers.get_list_of_customers()
#build df
df_buildings = get_buildings_df(client, customers)
#test
df_buildings.head()

,building_id,customerID,customer_name,address,city,atvise_display_name,base,heatingType,heatingSurface,access_key,nickname,postal_code,gui_name,coordinates,CustomerAlias,gfid,modular_system,LayoutID
0,756,66,HwS,Wolzogenstr.28,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,14163,Hilfswerk-Siedlung GmbH,"[52.432631, 13.234862]",None,GFP-12505,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--gas:2--heating-circuit:1--heating-circu...
1,757,66,HwS,Hagelberger Str. 26,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,10965,Hilfswerk-Siedlung GmbH,"[52.491932, 13.379842]",None,GFP-13438,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--gas:2--heating-circuit:1
2,758,66,HwS,Hochstr. 8,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,13357,Hilfswerk-Siedlung GmbH,"[52.54765, 13.381522]",None,GFP-12475,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--heating-circuit:1
3,759,66,HwS,Planufer 82a,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,10967,Hilfswerk-Siedlung GmbH,"[52.494745, 13.413277]",None,GFP-12530,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--heating-circuit:1
4,761,66,HwS,Gabainstr. 13,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,12247,Hilfswerk-Siedlung GmbH,"[52.436169, 13.360646]",None,GFP-12605,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--gas:2--global-separation-circuit:1--hea...


## get sub component 

In [11]:
df_buildings['sub_components'] = df_buildings['building_id'].apply(get_sub_component)

In [31]:
#df_buildings[['building_id','LayoutID','sub_components']].head()

## get sensor count

In [14]:
df_buildings['t_sensor_count'] = df_buildings['building_id'].apply(get_t_sensor_count)

In [15]:
#df_buildings.count()

In [16]:
# Initialize a set to collect unique key-value variations
unique_subcomponents = set()

# Loop through each dictionary in the column
for comp_dict in df_buildings['sub_components']:
    for k, v in comp_dict.items():
        variation = f"{k}: {v}"
        unique_subcomponents.add(variation)

# Display all unique variations
for variation in sorted(unique_subcomponents):
    print(variation)

b(n): type : BOILER sub_type : GAS
b(n): type : BOILER sub_type : NOT_SPECIFIED
b(n): type : BOILER sub_type : OIL
b(n): type : BOILER sub_type : PELLET
buffer(n): type : BUFFER sub_type : NOT_SPECIFIED
chp(n): type : CHP sub_type : NOT_SPECIFIED
dh: type : DISTRICT_HEATING sub_type : NOT_SPECIFIED
global: type : GLOBAL sub_type : NOT_SPECIFIED
hc(n): type : HEATING_CIRCUIT sub_type : FLOOR_HEATING
hc(n): type : HEATING_CIRCUIT sub_type : NOT_SPECIFIED
hc(n): type : HEATING_CIRCUIT sub_type : RADIATOR_HEATING
hp(n): type : HEAT_PUMP sub_type : AIR
hp(n): type : HEAT_PUMP sub_type : EXHAUST_AIR
hp(n): type : HEAT_PUMP sub_type : NOT_SPECIFIED
hw(n): type : WARM_WATER sub_type : FRESH_WATER_STATION
hw(n): type : WARM_WATER sub_type : NOT_SPECIFIED
hw(n): type : WARM_WATER sub_type : STORAGE_TANK
lh: type : LOCAL_HEATING_STATION sub_type : NOT_SPECIFIED
pv(n): type : PHOTOVOLTAIC sub_type : NOT_SPECIFIED
st(n): type : SOLAR_THERMAL sub_type : NOT_SPECIFIED


## get devices

In [17]:
df_devices=get_devices(client)
df_devices = df_devices.rename(columns={"type": "device_type"})

In [32]:
#df_devices.head()

In [33]:
#df_devices.count()

In [20]:
df_devices[['device_type','building_id']].groupby('device_type').count()

,building_id
device_type,
ECR_LW300,129
RUT956,970
RevPiConnectSE,1
RevPiCore32SE,3
WAGOPFC200,1


## Merge Dfs

In [21]:
# Merge the two DataFrames on building_id
df_merged = df_buildings.merge(
    df_devices[['building_id', 'device_type', 'device_id']],
    on='building_id',
    how='left'  # ensures all rows from df_buildings are kept
)

In [34]:
#df_merged.head()

In [35]:
#df_merged.count()

In [36]:
df_merged.columns

Index(['building_id', 'customerID', 'customer_name', 'address', 'city',
       'atvise_display_name', 'base', 'heatingType', 'heatingSurface',
       'access_key', 'nickname', 'postal_code', 'gui_name', 'coordinates',
       'CustomerAlias', 'gfid', 'modular_system', 'LayoutID', 'sub_components',
       't_sensor_count', 'device_type', 'device_id'],
      dtype='object')

## Exports df to csv

In [29]:
# get_buildings_df
if True:
    today_str = datetime.today().strftime('%Y-%m-%d')
    filename = f"Building_device_Layout_{today_str}.csv"
    filepath = os.path.join('OUTs', filename)
    df_merged[['building_id',
               'customerID',
               'customer_name',
               'address',
               'postal_code',
               'city',
               'device_type',
               'device_id',
               'LayoutID',
               'sub_components',
               'modular_system',
               't_sensor_count',
               #'atvise_display_name',
               #'base',
               #'coordinates',
               'gfid']].to_csv(filepath, index=False)